# Final Project, Milestone 2

In [62]:
URL="http://10.128.0.4:8080"

In [3]:
!trino $URL --execute "show catalogs"

"bigquery"
"mongo"
"mysql"
"postgres"
"system"
"tpch"


## Data Preparation

## Testing the databases

### Querying MySQL through Trino:

In [12]:
!trino $URL --execute "show schemas from mysql"

"final_project"
"information_schema"
"performance_schema"


In [15]:
!trino $URL --execute "show tables from mysql.final_project"

"shopper"


In [17]:
!trino $URL --execute "select * from mysql.final_project.shopper limit 3"

"1","Joseph","Lopez","Vincent-Lane","412 Kelly Rue Suite 279","Suite 984","Port Danielleville","Smithfurt","DC","07289","402-759-8598","(486)395-765","williamrobinson@example.org"
"2","Susan","Harvey","Smith PLC","07244 Thomas Common","Apt. 410","Justinland","Tinaview","SD","58509","969-886-3466","3479738356  ","rogerwelch@example.com"
"3","Kerri","Wilson","Bell-Johnson","775 Tina Tunnel Suite 964","Suite 178","Port Kenneth","Odonnellshire","AL","24610","+1-740-921-7","432-964-3447","sheribrown@example.com"


### Querying Postgres through Trino:

In [18]:
!trino $URL --execute "show schemas from postgres"

"final_project"
"information_schema"
"pg_catalog"
"public"


In [44]:
!trino $URL --execute "show tables from postgres.final_project"

"reservations"
"reviews"
"shopper"


In [20]:
!trino $URL --execute "select * from postgres.final_project.reservations limit 3"

"1","13","Wilson, Cooper and Brewer","and Sons","92579 Jamie Locks","Apt. 262","West Timothy","MO","64095","71.876170","-154.609189","EG","2023-11-27","2023-12-03","6243.60"
"2","20","Ramos Inc","Ltd","32601 Murray Alley Suite 713","Apt. 345","West Derekborough","RI","57910","59.807628","61.185516","BR","2023-11-21","2023-11-22","2363.32"
"3","3","Miller-Smith","and Sons","514 Walker Village Apt. 922","Apt. 975","Lozanostad","NE","39982","17.988926","-152.458124","BH","2023-12-22","2023-12-25","8309.81"


### Querying MongoDB through Trino:

In [4]:
!trino $URL --execute "show schemas from mongo"

"college"
"final_project"
"information_schema"


In [5]:
!trino $URL --execute "show tables from mongo.final_project"

"ticketing"


In [6]:
!trino $URL --execute "select * from mongo.final_project.ticketing limit 3"

"656aa6c648a03d23c924e3b4","c7ea34f4-ccb0-45cd-bd64-f920e72bd44c","25","Batik Air","5677","Helsinki Vantaa airport","Bologna airport","2023-11-16","1900-01-01 15:16:38","2023-11-20","1900-01-01 17:16:38","1","448","KHR"
"656aa6c648a03d23c924e3b5","8d17742f-816f-45fe-9da6-1c9d1a04fcd3","39","China United Airlines","9711","Philip S W Goldson International airport","Schiphol airport","2023-11-04","1900-01-01 12:43:36","2023-11-14","1900-01-01 19:43:36","2","774","SBD"
"656aa6c648a03d23c924e3b6","4bf97b83-fe10-4601-9f65-fa6ff0b4165b","30","S7 Airlines","1726","Nanning-Wuyu airport","Port Columbus International airport","2023-12-23","1900-01-01 08:56:23","2023-12-23","1900-01-01 14:56:23","2","915","MYR"


### Querying BigQuery through Trino:

In [25]:
!trino $URL --execute "show schemas from bigquery"

"cs327e_final_project_2023"
"information_schema"


In [26]:
!trino $URL --execute "show tables from bigquery.cs327e_final_project_2023"

"currency"
"reservations"
"ticketing"


In [27]:
!trino $URL --execute "select * from bigquery.cs327e_final_project_2023.currency limit 3"

"STD","Guatemalan quetzal","AL","Burundi"
"SVC","United States dollar","CV","Mexico"
"MNT","Uzbekistani soʻm","CV","Iran"


## Part 2: Federated Queries

Q1: Who are the shoppers who have reservations in a different city from their own city and who
paid for their reservations in Mongolian Tugrugs ~US dollars~ ? For those shoppers, return their cust_id, email, along with their reservation count. Order the results by cust_id. Limit the results to 5 records.

In [58]:
!trino $URL --execute "SELECT s.cust_id, s.email, COUNT(r.res_id) AS reservation_count \
FROM mysql.final_project.shopper s \
JOIN postgres.final_project.reservations r ON s.cust_id = r.cust_id \
JOIN mongo.final_project.ticketing t ON s.cust_id = t.cust_id \
JOIN bigquery.cs327e_final_project_2023.currency c ON t.curr_code = c.curr_code \
WHERE r.city <> s.city AND c.curr_code ='MNT' \
GROUP BY s.cust_id, s.email \
ORDER BY s.cust_id \
LIMIT 5; \
"
# was missing a backlash at end

"8","fischerlori@example.net","3"


Q2: Who are the shoppers who have bought non-stop tickets on ~American~ Shandong Airlines with a
departure date between 01/01/2020 and 12/31/2024? Return the shopper’s cust_id, email,
dep_airport and dep_date. Order the results by dep_date. Limit the results to 5 records.

In [75]:
!trino $URL --execute "SELECT s.cust_id, s.email, t.dep_airport, t.dep_date \
FROM mysql.final_project.shopper s \
JOIN mongo.final_project.ticketing t ON s.cust_id = t.cust_id \
WHERE t.airline = 'Shandong Airlines' AND t.stops = 'non-stop' AND CAST(t.dep_date AS date) BETWEEN DATE '2020-01-01' AND DATE '2024-12-31' \
ORDER BY CAST(t.dep_date AS date) \
LIMIT 5; \
"

"14","ijohnson@example.com","William P Hobby airport","2023-11-09"
"41","imoss@example.com","Chennai International airport","2023-11-26"
"48","douglasjasmine@example.org","Cataratas del Iguazu airport","2023-12-10"
"31","louis41@example.net","Macapa International airport","2023-12-30"


Q3: Which shoppers have reservations and tickets with matching dates such that the arr_date
and dep_date are equal between reservations and ticketing? For such shoppers, sum up their
pmt_amt and tik_amt and return this sum as trip_amount. Also, return the shoppers’ cust_id,
email, pmt_amt, tik_amt. Order the results by trip_amount in descending order. Limit the results
to 5 records.

In [76]:
!trino $URL --execute "SELECT s.cust_id, s.email, SUM(r.pmt_amt) + SUM(t.tik_amt) AS trip_amount, SUM(r.pmt_amt) AS pmt_amt, SUM(t.tik_amt) AS tik_amt \
FROM mysql.final_project.shopper s \
JOIN postgres.final_project.reservations r ON s.cust_id = r.cust_id \
JOIN mongo.final_project.ticketing t ON s.cust_id = t.cust_id \
WHERE CAST(r.dep_date as date) = CAST(t.arr_date as date) OR CAST(r.arr_date as date) = CAST(t.dep_date as date) \
GROUP BY s.cust_id, s.email \
ORDER BY trip_amount DESC \
LIMIT 5; \
"

"17","jamesstanton@example.org","13093.95","11824.95","1269"
"47","wgarcia@example.com","5902.32","3394.32","2508"
"46","taylorkent@example.org","4233.83","3648.83","585"
"21","ttyler@example.org","1498.62","1188.62","310"


## Part 3: Trino Evaluation

Conduct a more thorough evaluation of Trino. Think of other tests you can run to gain a more complete picture of Trino’s functionality. Come up with some tests and run them from your notebook. Test a variety of scenarios:
* DDL
* DML
* SELECT statements

### DDL (CREATE, ALTER, DROP)

**PostgreSQL: CREATE a new table that contains reviews/ratings from customers regarding their reservation stay.**

In [65]:
!trino $URL --execute "DROP TABLE IF EXISTS postgres.final_project.reviews;"

!trino $URL --execute "CREATE TABLE IF NOT EXISTS postgres.final_project.reviews (\
review_id INT,\
cust_id INT,\
review_comment VARCHAR,\
rating INT,\
review_date DATE\
);\
"
print("postgres.final_project.reviews successfully created.")

DROP TABLE
CREATE TABLE
postgres.final_project.reviews successfully created.


**MySQL: Add two new columns to mysql.final_project.shoppers that include a shopper's reward points for their booked reservations and flights.**

In [55]:
!trino $URL --execute "ALTER TABLE IF EXISTS mysql.final_project.shopper ADD COLUMN IF NOT EXISTS reservations_points INT;"

!trino $URL --execute "ALTER TABLE IF EXISTS mysql.final_project.shopper ADD COLUMN IF NOT EXISTS travel_points INT;"

ADD COLUMN
ADD COLUMN


### DML (INSERT, UPDATE, DELETE)

**mysql.final_project.reviews: Insert a new row**

In [69]:
!trino $URL --execute "INSERT INTO postgres.final_project.reviews (review_id, cust_id, review_comment, rating, review_date)\
VALUES (1, 1, 'The Cityscape Retreat''s impeccable cleanliness and convenient location near attractions make it a standout choice.', 5, CAST('2023-12-06' AS date));"  

INSERT: 1 row


**BigQuery: Inserts a new row into the currency table.**

In [49]:
!trino $URL --execute "INSERT INTO bigquery.cs327e_final_project_2023.currency (curr_code, curr_name, cntry_code, cntry_name)\
VALUES ('DRP', 'Darub pounds', 'PH', 'Philippines');"

INSERT: 1 row


**MongoDB: Deletes rows from mongo.final_project.ticketing collection that have "Spirit Airlines" as the airline.**

In [52]:
!trino $URL --execute "DELETE FROM mongo.final_project.ticketing WHERE airline = 'Spirit Airlines';"

DELETE: 1 row


## SELECT statements

In [70]:
# Show all data from the newly created reviews table in Postgres
!trino $URL --execute "SELECT * FROM postgres.final_project.reviews;"

"1","1","The Cityscape Retreat's impeccable cleanliness and convenient location near attractions make it a standout choice.","5","2023-12-06"


In [74]:
# For the first 5 records, display the new added columns along with cust_id
# There shouldn't be any values in the new columns because no data was added yet
!trino $URL --execute "SELECT cust_id, reservations_points, travel_points FROM mysql.final_project.shopper LIMIT 5;"

"1","",""
"2","",""
"3","",""
"4","",""
"5","",""


In [71]:
# Display all rows from the currency table whose currency code is DRP.
!trino $URL --execute "SELECT * FROM bigquery.cs327e_final_project_2023.currency WHERE curr_code = 'DRP';"

"DRP","Darub pounds","PH","Philippines"


In [72]:
# Checking to see if there's any ticketing data whose airline is Spirit Airlines
# There should be no data returned because all these rows were previously deleted
!trino $URL --execute "SELECT * FROM mongo.final_project.ticketing WHERE airline = 'Spirit Airlines';"

### Trino Findings Summary

After testing a variety of Trino scenarios including DDL, DML, and SELECT statements, it was found that data spanning various types of databases could be accessed and modified. The **CREATE TABLE** SQL statement against the PostgreSQL final_project schema resulted in a new table named “reviews”, which included new data that was inserted using an **INSERT INTO** statement. It was also seen that new columns, “reservations_points” and “travel_points”, were added to the MySQL final_project schema, represented by the **ALTER TABLE** statement. Although this Trino connector didn’t support **UPDATE**, it supported **INSERT** and **DELETE**, represented by the **INSERT INTO** statement against the BigQuery final_project currency table, resulting in a new row containing a new currency. The **DELETE FROM** statement against the MongoDB final_project ticketing collection resulted in deletion of all rows satisfying a certain criterion (Spirit Airlines), which represents how Trino successfully supports DML scenarios. Lastly, after running a **SELECT** statement to verify each DDL and DML scenario, it was seen that the newly inserted data was successfully returned (Postgres and BigQuery), new columns were shown (MySQL), and no data was returned (MongoDB). 